In [184]:
import pandas as pd

properties_df = pd.read_json('data/property_listings.json')
properties_df.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   page           377 non-null    int64 
 1   location       377 non-null    object
 2   neighborhood   377 non-null    object
 3   area           377 non-null    object
 4   price          377 non-null    object
 5   fee            377 non-null    object
 6   bedrooms       377 non-null    object
 7   bathrooms      377 non-null    object
 8   parking_spots  377 non-null    object
 9   description    377 non-null    object
 10  url            377 non-null    object
dtypes: int64(1), object(10)
memory usage: 32.5+ KB


In [168]:
embeddings_df = pd.read_csv('data/properties_with_embeddings.csv')
embeddings_df['property_string'][embeddings_df['property_string'].str.contains('san rafael')].iloc[0]

'Ubicación: urbanización san rafael, Barrio: Alangasí, Quito, Area: 275 m² tot., Precio: USD 125.900, Alícuota: USD 60 Alícuota, Habitaciones: 3 hab., Baños: 2 baños, Parqueaderos: 2 estac., Descripción: Venga a vivir en una casa de 3 habitaciones en una urbanización privada de San Rafael. La casa forma parte de un pequeño conjunto de 4 viviendas. Dormitorio principal muy espacioso y luminoso, que cuenta con amplio clóset y su baño privado. Las 2 habitaciones secundarias comparten un baño. Una de las dos habitaciones tiene acceso a una pequeña terraza, a través del ventanal. La otra habitación tiene luz y calor gracias al ventanal en uno de los lados. Área social en la planta baja. Medio baño para visitas. Pequeña bodega bajo las gradas de la vivienda. La cocina tiene además al lado, un espacio para las máquinas de lavado y secado de ropa. El comedor de la casa, se comunica con el patio. El patio trasero tiene área dura en cerámica y espacio verde. En la parte delantera de la casa tamb

In [183]:
properties_df[properties_df['url'] == 'plusvalia.com/propiedades/clasificado/casa-remodelada-en-conjunto-sector-colegio-einstein-143791680.html']
properties_df.iloc[268]

page                                                             9
location                                       cesar borja cordero
neighborhood                                       Carcelén, Quito
area                                                   120 m² tot.
price                                                   USD 91.000
fee                                                USD 25 Alícuota
bedrooms                                                    3 hab.
bathrooms                                                  2 baños
parking_spots                                             2 estac.
description      Bonita casa remodelada dentro de conjunto sect...
url              plusvalia.com/propiedades/clasificado/veclcain...
Name: 268, dtype: object

In [178]:
carcelen = embeddings_df['property_string'][embeddings_df['property_string'].str.contains('Carcelén')]
carcelen.iloc[6]

'Ubicación: URBANIZACIÓN BALCÓN DEL NORTE, Barrio: Carcelén, Quito, Area: 90 m² tot., Precio: USD 112.500, Alícuota: , Habitaciones: 2 hab., Baños: 1 baño, Parqueaderos: 1 estac., Descripción: Encuentra el espacio ideal que mejor se adapte a tus gustos y necesidades. Departamentos de: 2 dormitorios se encuentra en un primer piso. sala-comedor. cocina estilo americana. area de lavado. 2 dormitorios que comparten. 1 baño completo Y que hace las veces de baño social. 1 estacionamiento. en la Urbanización balcón del norte por el sector de Carcelén Alto. Ubicaciones estratégicas. Áreas comunales llenas de arboles con senderos para caminar y canchas deportivas. Jardines. La paz que buscas para tu hogar, ¡La encuentras en Edificio Portofino. Vive rodeado de áreas verdes naturales y árboles sembrados en toda la Urbanización. Encuentra la dualidad perfecta entre campo y ubicación estratégica. Además, el edificio cuenta con terraza, para que descanses al aire libre y una sala de reuniones. Estos

In [224]:
import json
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('spanish'))

with open('data/property_listings.json', 'r') as file:
    properties = json.load(file)

properties[0]


[nltk_data] Downloading package stopwords to /Users/juank/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'page': 1,
 'location': 'Las Cucardas',
 'neighborhood': 'El Inca, Quito',
 'area': '145 m² tot.',
 'price': 'USD 450',
 'fee': 'USD 1 Alícuota',
 'bedrooms': '2 estac.',
 'bathrooms': 'N/A',
 'parking_spots': 'N/A',
 'description': 'Arriendo Local comercial - bodega de 145m2 en el Sector de El Inca, es apta para taller de confección textil, serigrafía, taller mecánico, taller de motos, área de logística, transporte, transporte de encomiendas, archivo, almacenamiento de materiales eléctricos, madera, muebles, insumos textiles, repuestos, llantas, productos no perecibles, etc. Excelente ubicación con salida hacia la Av. El Inca, Av. 6 de Diciembre. Descripción: El local - bodega tiene 145 m2 ubicado en primer piso con un baño y dos estacionamientos, área para carga y descarga de mercadería, seguridad en su ingreso. Medidor independiente de luz. precio: $450 y 2 meses de Garantía. citas: Ver datos cbr Eveling Rosas 0 9 8 7 0 0 6 8 0 6. #bodegadearriendo, #rentobodegaelinca, #galpon, #bo

In [226]:
def clean_and_transform_data(data):
  """
  This function cleans and transforms a real estate data dictionary.

  Args:
      data (dict): The real estate data dictionary.

  Returns:
      dict: The updated dictionary with cleaned and transformed data.
  """
 # Create a new dictionary to store cleaned data
  cleaned_data = {}

  # Copy all keys except "page" and "url"
  for key, value in data.items():
    if key not in ["page", "url"]:
      cleaned_data[key] = value

  # Clean description: normalize, remove stop words, and chunk
  cleaned_data['description'] = clean_text(data["description"])
  cleaned_data["description"] = chunk_description(cleaned_data["description"])  # Pass cleaned description

  # Extract and clean numerical values (bedrooms, bathrooms, parking spots)
  cleaned_data["bedrooms"] = int(re.findall(r"\d+", data["bedrooms"])[0])
  cleaned_data["bathrooms"] = int(re.findall(r"\d+", data["bathrooms"])[0]) if data['bathrooms'] != 'N/A' else None
  cleaned_data["parking_spots"] = int(re.findall(r"\d+", data["parking_spots"])[0]) if data["parking_spots"] != 'N/A' else None  # Handle missing values

  # Clean location and neighborhood (directly using clean_text)
  cleaned_data["location"] = clean_text(data["location"])
  cleaned_data["neighborhood"] = clean_text(data["neighborhood"])

  return cleaned_data

def clean_text(text):
  """
  This function cleans text data by normalizing and removing stop words.

  Args:
      text (str): The text string to be cleaned.

  Returns:
      str: The cleaned text string.
  """
  text = text.lower()  # Normalize to lowercase
  text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
  # Remove stop words
  text = " ".join([word for word in text.split() if word not in stop_words])
  return text

def chunk_description(description):
  """
  This function splits the description text into sentences.

  Args:
      description (str): The description text of the property.

  Returns:
      list: A list of sentences from the description.
  """
  sentences = re.split(r'[.!?]+', description)
  return sentences

In [227]:
clean_example = clean_and_transform_data(properties[0])
clean_example

[]


{'location': 'cucardas',
 'neighborhood': 'inca quito',
 'area': '145 m² tot.',
 'price': 'USD 450',
 'fee': 'USD 1 Alícuota',
 'bedrooms': 2,
 'bathrooms': None,
 'parking_spots': None,
 'description': ['arriendo local comercial bodega 145m2 sector inca apta taller confección textil serigrafía taller mecánico taller motos área logística transporte transporte encomiendas archivo almacenamiento materiales eléctricos madera muebles insumos textiles repuestos llantas productos perecibles etc excelente ubicación salida hacia av inca av 6 diciembre descripción local bodega 145 m2 ubicado primer piso baño dos estacionamientos área carga descarga mercadería seguridad ingreso medidor independiente luz precio 450 2 meses garantía citas ver datos cbr eveling rosas 0 9 8 7 0 0 6 8 0 6 bodegadearriendo rentobodegaelinca galpon bodegaarchivo bodegaalmacenamiento localparabodegadearriendo wescobodegaelinca arriendolocalparatallertextil bodegagym arriendoparaencomiendas arriendoparatransporte tallerd

In [139]:
mask = properties_df['url'] == 'plusvalia.com/propiedades/clasificado/veclcain-venta-casa-en-una-de-las-mejores-urbanizaciones-del-66367428.html'
print(properties_df[mask]['description'].values[0])

Venga a vivir en una casa de 3 habitaciones en una urbanización privada de San Rafael. La casa forma parte de un pequeño conjunto de 4 viviendas. Dormitorio principal muy espacioso y luminoso, que cuenta con amplio clóset y su baño privado. Las 2 habitaciones secundarias comparten un baño. Una de las dos habitaciones tiene acceso a una pequeña terraza, a través del ventanal. La otra habitación tiene luz y calor gracias al ventanal en uno de los lados. Área social en la planta baja. Medio baño para visitas. Pequeña bodega bajo las gradas de la vivienda. La cocina tiene además al lado, un espacio para las máquinas de lavado y secado de ropa. El comedor de la casa, se comunica con el patio. El patio trasero tiene área dura en cerámica y espacio verde. En la parte delantera de la casa también se cuenta con área verde. Estacionamiento para 2 vehículos. Amplio acceso desde la calle hasta el lugar de la casa. Urbanización con seguridad 24/7. Servicios completos. Áreas verdes muy grandes. Canc

In [87]:
properties_df = properties_df[properties_df['page'] < 14]
duplicates = properties_df[properties_df.loc[:, 'location':].duplicated()]
duplicates.count()

page             48
location         48
neighborhood     48
area             48
price            48
fee              48
bedrooms         48
bathrooms        48
parking_spots    48
description      48
url              48
dtype: int64

In [89]:
properties_df['location'].describe()
properties_df[properties_df['location'] == 'Urbanización El Condado']

,page,location,neighborhood,area,price,fee,bedrooms,bathrooms,parking_spots,description,url
109,4,Urbanización El Condado,N/A,122 m² tot.,USD 129.900,N/A,2 baños,2 estac.,N/A,N/A,plusvalia.comN/A
152,6,Urbanización El Condado,"El Condado, Quito",122 m² tot.,USD 129.900,USD 145 Alícuota,2 baños,2 estac.,N/A,¡excelente oportunidad! En la Urb. El Condado ...,plusvalia.com/propiedades/clasificado/veclapin...
325,11,Urbanización El Condado,N/A,188 m² tot.,USD 155.000,N/A,3 hab.,2 estac.,N/A,N/A,plusvalia.comN/A
348,12,Urbanización El Condado,N/A,356 m² tot.,USD 264.900,N/A,4 hab.,4 baños,4 estac.,N/A,plusvalia.comN/A


In [90]:
duplicates[duplicates['location'] == 'Urbanización El Condado']

,page,location,neighborhood,area,price,fee,bedrooms,bathrooms,parking_spots,description,url


In [ ]:
duplicates

In [74]:
duplicates[duplicates['location'] == 'cashapamba']

,page,location,neighborhood,area,price,fee,bedrooms,bathrooms,parking_spots,description,url
32,2,cashapamba,"Sangolqui, Quito",2345 m² tot.,USD 258.000,USD 20 Alícuota,2 hab.,3 baños,6 estac.,Linda quinta terreno 2345m2 dentro de conjunto...,plusvalia.com/propiedades/clasificado/veclhqin...


,page,location,neighborhood,area,price,fee,bedrooms,bathrooms,parking_spots,description,url
408,16,Ponceano,N/A,115 m² tot.,USD 87.000,N/A,3 hab.,2 baños,2 estac.,N/A,plusvalia.comN/A
409,16,moran,N/A,130 m² tot.,USD 47.000,N/A,3 hab.,2 baños,1 estac.,N/A,plusvalia.comN/A
410,16,Calderón Av. Carlos Mantilla,N/A,140 m² tot.,USD 50.000,N/A,3 hab.,2 baños,1 estac.,N/A,plusvalia.comN/A
411,16,Eloy Alfaro y Gaspar de Villarroel,N/A,400 m² tot.,USD 200.000,N/A,4 hab.,4 baños,2 estac.,N/A,plusvalia.comN/A
412,16,calle de entrada a Llano Grande García Moreno ...,N/A,126 m² tot.,USD 47.000,N/A,4 hab.,3 baños,1 estac.,N/A,plusvalia.comN/A
413,16,Santa Eulalia,N/A,2078 m² tot.,USD 78.900,N/A,N/A,N/A,N/A,N/A,plusvalia.comN/A
414,16,Pomasqui,N/A,120 m² tot.,USD 105.000,N/A,3 hab.,2 baños,2 estac.,N/A,plusvalia.comN/A
415,16,republica y pradera,N/A,135 m² tot.,USD 79.000,N/A,2 hab.,3 baños,1 estac.,N/A,plusvalia.comN/A
416,16,N12 BENJAMIN CARRION - E9-,N/A,267 m² tot.,USD 99.000,N/A,3 hab.,3 baños,2 estac.,N/A,plusvalia.comN/A
417,16,Alangasi,N/A,763 m² tot.,USD 290.000,N/A,4 hab.,7 baños,5 estac.,N/A,plusvalia.comN/A


,page,location,neighborhood,area,price,fee,bedrooms,bathrooms,parking_spots,description,url
31,2,Gonzalez Súarez,"González Suárez, Quito",344 m² tot.,USD 2.262,USD 1 Alícuota,3 hab.,4 baños,4 estac.,"Valor renta $2262, 00 (incluye alícuota) Depar...",plusvalia.com/propiedades/clasificado/alclapin...


In [21]:
property = properties_df[properties_df['price'].str.contains('USD 127.500')]
property

,page,location,neighborhood,area,price,fee,bedrooms,bathrooms,parking_spots,description
149,6,urbanización san rafael,N/A,275 m² tot.,USD 127.500,N/A,3 hab.,2 baños,2 estac.,N/A


In [16]:
property_description = property['description']
property_description

'En el corazón del Valle de los Chillos vendo hermosa casa con amplios y elegantes espacios ideal para distrutarla en familia, esta dentro de urbanización qué tiene áreas verdes, parques, área para patinaje, excelente entorno, clima, ubicación. La casa está distribuida de la siguiente manera: Amplio jardín de entrada. Sala comedor amplios. Cocina con family room. 3 habitaciones amplias con baño completo. 1 habitación máster super amplia con ventanales de pared a techo, closet y walking closet. Todos los muebles son personalizados y funcionales, la casa está domotizada. Área de lavandería. Bodega muy bien dispuesta. Baño completo de servicio. Baño de visitas completo. Piscina temperada con iluminación. Patio área de bbq. Sala familia con mesa de Poker con usb, Bar, baño completo, con acceso directo al jardín y a la piscina. Estacionamiento para 5 vehículos. Un propiedad con mucho estilo. Ideal para usted'